In [81]:
# Dependencies
import numpy as np
import tweepy
import time
import json
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [82]:
# Twitter Credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [83]:
#Selected news outlets for analysis
target_user=("@BBCNews", "@CNNbrk", "@CBSNews","@FoxNews", "@nytimes")

# Counter
counter = 1

# List for dictionaries of results
results_list = []

# Variable for max_id
oldest_tweet = None

# Loop through 5 pages of tweets (total 100 tweets)
for x in range(20):

    # Get all tweets from home feed
    public_tweets = api.user_timeline(target_user, max_id = oldest_tweet)

# Loop through each user
for user in target_user:

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through pages of tweets (total 100 tweets)
    for x in range(1, 50):

        # Get all tweets from home feed
        public_tweets = api.user_timeline(user, page=x)
        
        # Loop through all tweets
        for tweet in public_tweets:

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]

            # Add each value to the appropriate list
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)

    # Create a dictionaty of results
    user_results = {
        "News Organization": user,
        "Date": tweet["created_at"],
        "Compound Score": np.mean(compound_list),
        "Postive Score": np.mean(positive_list),
        "Neutral Score": np.mean(neutral_list),
        "Negative Score": np.mean(negative_list)
    }

    # Append dictionary to list
    results_list.append(user_results)

    # Print the Averages
    print(f"User: {user}")
    print(f"Compound: {np.mean(compound_list):.3f}")
    print(f"Positive: {np.mean(positive_list):.3f}")
    print(f"Neutral: {np.mean(neutral_list):.3f}")
    print(f"Negative: {np.mean(negative_list):.3f}")

User: @BBCNews
Compound: 0.011
Positive: 0.078
Neutral: 0.838
Negative: 0.084
User: @CNNbrk
Compound: -0.116
Positive: 0.056
Neutral: 0.845
Negative: 0.098
User: @CBSNews
Compound: -0.085
Positive: 0.064
Neutral: 0.829
Negative: 0.107
User: @FoxNews
Compound: 0.006
Positive: 0.078
Neutral: 0.840
Negative: 0.082
User: @nytimes
Compound: 0.016
Positive: 0.077
Neutral: 0.851
Negative: 0.072


In [84]:
len(public_tweets)

20

In [85]:
# Print tweet in JSON
print(json.dumps(public_tweets, sort_keys=True, indent=4, separators=(',', ': ')))

[
    {
        "contributors": null,
        "coordinates": null,
        "created_at": "Fri Jul 06 09:14:01 +0000 2018",
        "entities": {
            "hashtags": [],
            "symbols": [],
            "urls": [
                {
                    "display_url": "twitter.com/i/web/status/1\u2026",
                    "expanded_url": "https://twitter.com/i/web/status/1015161936739209216",
                    "indices": [
                        117,
                        140
                    ],
                    "url": "https://t.co/ij95wj2q4m"
                }
            ],
            "user_mentions": []
        },
        "favorite_count": 179,
        "favorited": false,
        "geo": null,
        "id": 1015161936739209216,
        "id_str": "1015161936739209216",
        "in_reply_to_screen_name": null,
        "in_reply_to_status_id": null,
        "in_reply_to_status_id_str": null,
        "in_reply_to_user_id": null,
        "in_reply_to_user_id_str": null

In [86]:
# Create DataFrame from Results List
results_df = pd.DataFrame(results_list).set_index("News Organization").round(3)
results_df

,Compound Score,Date,Negative Score,Neutral Score,Postive Score
News Organization,,,,,
@BBCNews,0.011,Thu Jul 05 05:40:29 +0000 2018,0.084,0.838,0.078
@CNNbrk,-0.116,Mon Apr 30 10:16:52 +0000 2018,0.098,0.845,0.056
@CBSNews,-0.085,Fri Jul 06 11:11:23 +0000 2018,0.107,0.829,0.064
@FoxNews,0.006,Tue Jul 10 15:29:14 +0000 2018,0.082,0.840,0.078
@nytimes,0.016,Fri Jul 06 03:47:02 +0000 2018,0.072,0.851,0.077


In [87]:
#Export the dataframe to csv file
results_df.to_csv('Twitter_News Organization Sentiments.csv', encoding='utf-8', index=False)

In [88]:
#"@BBCNews", "@CNNbrk", "@CBSNews","@FoxNews", "@nytimes"
bbc_data = results_df.loc[results_df["News Organization"] == "@BBC",:]
cnn_data = results_df.loc[results_df["News Organization"] == "@CNN",:]
cbs_data = results_df.loc[results_df["News Organization"] == "@CBSNews",:]
fox_data = results_df.loc[results_df["News Organization"] == "@FOXNews",:]
nytimes_data = results_df.loc[results_df["News Organization"] == "@nytimes",:]

KeyError: 'News Organization'

In [ ]:
# Create plot of the last 100 tweets from each news organization

sort_news=results_df.sort_values("Tweets Ago")


for i in np.arange(0,len(target_user)):
    news_org=results_df.loc[results_df["News Organization"]==target_user[i]]
    y_vals = results_df["Compound"]
    plt.plot(x_vals, y_vals, marker="o", linewidth=0.5, alpha=0.8)
    news_org["Compound"], color=colors[[i]]

# # Incorporate the other graph properties
now = datetime.now()
now = now.strftime("%Y-%m-%d %H:%M")
plt.title(f"Tweet Sentiment Analysis ({now}) for {target_us}")
plt.xlim([x_vals.max(),x_vals.min()]) #Bonus
plt.ylabel("Tweet Polarity")
plt.xlabel("Tweets Ago")
plt.show()

In [ ]:
#Plot bar graph of overall sentiments of the last 100 tweets

now=datetime.now()

for i in np.arange(0,len(target_user)):
    news_data = results_df.loc[results_df['User']==target_user[i]]
    Compound[target_user[i]] = news_data['Compound Score'].mean()
    tick_locations.append(i)
    
x_axis = np.arange(0, len(target_user) )
plt.xlim(len(target_user))
plt.ylim(min(cmpd_avg.values()), max(cmpd_avg.values()))
plt.title(f"Overall Twitter Sentiments from News Organizations {datetime.now().strftime('%Y-%m-%d %H:%M')}" )
plt.ylabel("Tweet Sentiment Value")
plt.xticks(tick_locations, cmpd_avg)
bar_graphs = plt.bar(x_axis, cmpd_avg.values(), alpha=0.75)